### Work Genre based and recomend a user based on genre

In [1]:
import pandas as pd
from pathlib import Path
# Import dependencies 
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
# from scipy.sparse import csr_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler
import hvplot.pandas

import tensorflow as tf
import warnings
warnings.simplefilter("ignore")

from matplotlib import pyplot as plt
from pprint import pprint
from sklearn.cluster import DBSCAN
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
links_csv = Path('Resources/ml-latest-small/links.csv')
ratings_csv = Path('Resources/ml-latest-small/ratings.csv')
movies_csv = Path('Resources/ml-latest-small/movies.csv')
tags_csv = Path('Resources/ml-latest-small/tags.csv')

In [3]:
tags_df = pd.read_csv(tags_csv)
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [4]:
tags_df.dtypes

userId        int64
movieId       int64
tag          object
timestamp     int64
dtype: object

In [5]:
year_tagged = 1970 + (tags_df['timestamp'] / 31540000)
year_tagged = year_tagged.astype('int')
year_tagged

0       2015
1       2015
2       2015
3       2015
4       2015
        ... 
3678    2007
3679    2007
3680    2017
3681    2017
3682    2017
Name: timestamp, Length: 3683, dtype: int32

In [6]:
tags_df['year_tagged'] = year_tagged
tags_df.head(25)

,userId,movieId,tag,timestamp,year_tagged
0,2,60756,funny,1445714994,2015
1,2,60756,Highly quotable,1445714996,2015
2,2,60756,will ferrell,1445714992,2015
3,2,89774,Boxing story,1445715207,2015
4,2,89774,MMA,1445715200,2015
5,2,89774,Tom Hardy,1445715205,2015
6,2,106782,drugs,1445715054,2015
7,2,106782,Leonardo DiCaprio,1445715051,2015
8,2,106782,Martin Scorsese,1445715056,2015
9,7,48516,way too long,1169687325,2007


In [7]:
ratings_df = pd.read_csv(ratings_csv)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
year_rated = 1970 + (ratings_df['timestamp'] / 31540000)
year_rated = year_rated.astype('int')
year_rated

0         2000
1         2000
2         2000
3         2000
4         2000
          ... 
100831    2017
100832    2017
100833    2017
100834    2017
100835    2017
Name: timestamp, Length: 100836, dtype: int32

In [9]:
ratings_df['year_rated'] = year_rated
ratings_df.head(25)

,userId,movieId,rating,timestamp,year_rated
0,1,1,4.0,964982703,2000
1,1,3,4.0,964981247,2000
2,1,6,4.0,964982224,2000
3,1,47,5.0,964983815,2000
4,1,50,5.0,964982931,2000
5,1,70,3.0,964982400,2000
6,1,101,5.0,964980868,2000
7,1,110,4.0,964982176,2000
8,1,151,5.0,964984041,2000
9,1,157,5.0,964984100,2000


In [10]:
ratings_df.loc[ratings_df['userId'] == 2]

,userId,movieId,rating,timestamp,year_rated
232,2,318,3.0,1445714835,2015
233,2,333,4.0,1445715029,2015
234,2,1704,4.5,1445715228,2015
235,2,3578,4.0,1445714885,2015
236,2,6874,4.0,1445714952,2015
237,2,8798,3.5,1445714960,2015
238,2,46970,4.0,1445715013,2015
239,2,48516,4.0,1445715064,2015
240,2,58559,4.5,1445715141,2015
241,2,60756,5.0,1445714980,2015


In [11]:
# Movie ID's that user 2 rated
ratings_df.loc[ratings_df['userId'] == 2]['movieId'].unique()

array([   318,    333,   1704,   3578,   6874,   8798,  46970,  48516,
        58559,  60756,  68157,  71535,  74458,  77455,  79132,  80489,
        80906,  86345,  89774,  91529,  91658,  99114, 106782, 109487,
       112552, 114060, 115713, 122882, 131724], dtype=int64)

In [12]:
# Total number user 2 rated
ratings_df.loc[ratings_df['userId'] == 2]['movieId'].nunique()

29

In [13]:
# Movie ID's that user 2 tagged
tags_df.loc[tags_df['userId'] == 2]['movieId'].unique()

array([ 60756,  89774, 106782], dtype=int64)

In [14]:
tags_df.loc[tags_df['userId'] == 2]['movieId'].nunique()

3

In [15]:
tags_df['tag'].value_counts()

tag
In Netflix queue     131
atmospheric           36
thought-provoking     24
superhero             24
funny                 23
                    ... 
small towns            1
In Your Eyes           1
Lloyd Dobbler          1
weak plot              1
Heroic Bloodshed       1
Name: count, Length: 1589, dtype: int64

In [16]:
ratings_df['year_rated'].value_counts()

year_rated
2000    10104
2017     7972
2007     7306
2016     6776
2018     6725
2015     6538
1996     5917
2005     5798
2012     4611
2008     4284
2009     4180
2003     4010
2006     3806
2002     3758
2001     3639
2004     3275
1999     2395
2010     2350
1997     2035
2011     1752
2013     1696
2014     1398
1998      511
Name: count, dtype: int64

In [17]:
ratings_df['year_rated'].unique()

array([2000, 2015, 2011, 2001, 1999, 1996, 2005, 2006, 2007, 2003, 2016,
       1998, 2009, 2017, 2013, 2012, 2018, 2014, 2010, 1997, 2004, 2002,
       2008])

In [18]:
tags_df['year_tagged'].value_counts()

year_tagged
2006    1533
2018     844
2016     355
2017     329
2015     195
2009     166
2010     132
2012      47
2007      46
2011      14
2013      10
2008       9
2014       3
Name: count, dtype: int64

In [19]:
movies_df = pd.read_csv(movies_csv)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
links_df = pd.read_csv(links_csv)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [21]:
movies_df.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [22]:
movies_df['genres'].nunique()

951

In [23]:
movies_df['genres'].head()

0    Adventure|Animation|Children|Comedy|Fantasy
1                     Adventure|Children|Fantasy
2                                 Comedy|Romance
3                           Comedy|Drama|Romance
4                                         Comedy
Name: genres, dtype: object

In [24]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
movies_df[['title', 'release_year']] = movies_df['title'].str.split(r' \s*\(\s*|\s*\)\s*', expand = True).iloc[:, [0, 1]]
movies_df.head()

,movieId,title,genres,release_year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [26]:
movies_df2 = movies_df.copy()
movies_df2.head()

,movieId,title,genres,release_year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [27]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|') )
movies_df.head()

,movieId,title,genres,release_year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [28]:
movies_df['genres'].value_counts()

genres
[Drama]                                                       1053
[Comedy]                                                       946
[Comedy, Drama]                                                435
[Comedy, Romance]                                              363
[Drama, Romance]                                               349
                                                              ... 
[Action, Crime, Horror, Mystery, Thriller]                       1
[Adventure, Animation, Children, Comedy, Musical, Romance]       1
[Action, Adventure, Animation, Comedy, Crime, Mystery]           1
[Children, Comedy, Fantasy, Sci-Fi]                              1
[Action, Animation, Comedy, Fantasy]                             1
Name: count, Length: 951, dtype: int64

In [29]:
#counting the most frequent genre in our dataset
from collections import Counter
genre_frequency = Counter(g for genres in movies_df['genres'] for g in genres)
print(f"There are {len(genre_frequency)} genres.")
genre_frequency

There are 20 genres.


Counter({'Drama': 4361,
         'Comedy': 3756,
         'Thriller': 1894,
         'Action': 1828,
         'Romance': 1596,
         'Adventure': 1263,
         'Crime': 1199,
         'Sci-Fi': 980,
         'Horror': 978,
         'Fantasy': 779,
         'Children': 664,
         'Animation': 611,
         'Mystery': 573,
         'Documentary': 440,
         'War': 382,
         'Musical': 334,
         'Western': 167,
         'IMAX': 158,
         'Film-Noir': 87,
         '(no genres listed)': 34})

In [30]:
merged_movie_ratings = pd.merge(movies_df, ratings_df, on='movieId', how='inner')
merged_movie_ratings.head()

,movieId,title,genres,release_year,userId,rating,timestamp,year_rated
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1,4.0,964982703,2000
1,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,5,4.0,847434962,1996
2,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,7,4.5,1106635946,2005
3,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,15,2.5,1510577970,2017
4,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,17,4.5,1305696483,2011


In [31]:
merged_movie_ratings.isnull().sum()

movieId          0
title            0
genres           0
release_year    17
userId           0
rating           0
timestamp        0
year_rated       0
dtype: int64

In [32]:
bool_series = pd.isnull(merged_movie_ratings["release_year"])
merged_movie_ratings[bool_series]

,movieId,title,genres,release_year,userId,rating,timestamp,year_rated
81292,40697,Babylon 5,[Sci-Fi],None,210,4.0,1527266191,2018
81293,40697,Babylon 5,[Sci-Fi],None,528,0.5,1391736467,2014
99325,140956,Ready Player One,"[Action, Sci-Fi, Thriller]",None,209,3.5,1524522239,2018
99326,140956,Ready Player One,"[Action, Sci-Fi, Thriller]",None,305,4.5,1533043577,2018
99327,140956,Ready Player One,"[Action, Sci-Fi, Thriller]",None,380,3.0,1534176158,2018
99328,140956,Ready Player One,"[Action, Sci-Fi, Thriller]",None,514,3.0,1533946119,2018
99459,143410,Hyena Road,[(no genres listed)],None,448,2.0,1474224802,2016
99526,147250,The Adventures of Sherlock Holmes and Doctor W...,[(no genres listed)],None,105,5.0,1526207354,2018
99617,149334,Nocturnal Animals,"[Drama, Thriller]",None,306,3.0,1518380372,2018
99799,156605,Paterson,[(no genres listed)],None,567,4.5,1525283448,2018


In [33]:
bool_series = pd.isnull(movies_df["release_year"])
movies_df[bool_series]

,movieId,title,genres,release_year
6059,40697,Babylon 5,[Sci-Fi],None
9031,140956,Ready Player One,"[Action, Sci-Fi, Thriller]",None
9091,143410,Hyena Road,[(no genres listed)],None
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,[(no genres listed)],None
9179,149334,Nocturnal Animals,"[Drama, Thriller]",None
9259,156605,Paterson,[(no genres listed)],None
9367,162414,Moonlight,[Drama],None
9448,167570,The OA,[(no genres listed)],None
9514,171495,Cosmos,[(no genres listed)],None
9515,171631,Maria Bamford: Old Baby,[(no genres listed)],None


In [34]:
merged_movie_ratings.head()

,movieId,title,genres,release_year,userId,rating,timestamp,year_rated
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1,4.0,964982703,2000
1,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,5,4.0,847434962,1996
2,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,7,4.5,1106635946,2005
3,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,15,2.5,1510577970,2017
4,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,17,4.5,1305696483,2011


In [35]:
movies_df2

,movieId,title,genres,release_year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018


In [36]:
movies_df2['genres'][0]

'Adventure|Animation|Children|Comedy|Fantasy'

In [37]:
## Get user input for the movie title
#search_title = input("Please enter the movie title you want to search for: ")
# Find the movie ID based on the title
#movie_id = [key for key, value in movie_titles.items() if value.lower() == search_title.lower()]
#if movie_id:
#    movie_id = movie_id[0]  # Get the first match
#    print(f"The movie ID for '{search_title}' is: {movie_id}")

In [38]:
movies_df.head()

,movieId,title,genres,release_year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [39]:
genres = set()
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        genres.add(genre)
genres

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [40]:
#counting the most frequent genre in our dataset 
from collections import Counter

genre_frequency = Counter(g for genres in movies_df['genres'] for g in genres)

print(f"There are {len(genre_frequency)} genres.")

genre_frequency

There are 20 genres.


Counter({'Drama': 4361,
         'Comedy': 3756,
         'Thriller': 1894,
         'Action': 1828,
         'Romance': 1596,
         'Adventure': 1263,
         'Crime': 1199,
         'Sci-Fi': 980,
         'Horror': 978,
         'Fantasy': 779,
         'Children': 664,
         'Animation': 611,
         'Mystery': 573,
         'Documentary': 440,
         'War': 382,
         'Musical': 334,
         'Western': 167,
         'IMAX': 158,
         'Film-Noir': 87,
         '(no genres listed)': 34})

In [41]:
keys = list(genre_frequency.keys())
values = list(genre_frequency.values())

In [42]:
cutoff = 200
genres = {'other':0}
b = 0
for k, v in zip(keys, values):
    if v < cutoff:
        genres['other'] += v
    else:
        genres[k] = v
print(genres)

{'other': 446, 'Adventure': 1263, 'Animation': 611, 'Children': 664, 'Comedy': 3756, 'Fantasy': 779, 'Romance': 1596, 'Drama': 4361, 'Action': 1828, 'Crime': 1199, 'Thriller': 1894, 'Horror': 978, 'Mystery': 573, 'Sci-Fi': 980, 'War': 382, 'Musical': 334, 'Documentary': 440}


In [43]:
len(genres)

17

In [44]:
genres = set(g for G in movies_df['genres'] for g in G)
genres_list = list(genres)
genres_list.pop(8)
len(genres_list)

19

In [45]:
genre_frequency = Counter(g for genres in movies_df['genres'] for g in genres)
genre_frequency

Counter({'Drama': 4361,
         'Comedy': 3756,
         'Thriller': 1894,
         'Action': 1828,
         'Romance': 1596,
         'Adventure': 1263,
         'Crime': 1199,
         'Sci-Fi': 980,
         'Horror': 978,
         'Fantasy': 779,
         'Children': 664,
         'Animation': 611,
         'Mystery': 573,
         'Documentary': 440,
         'War': 382,
         'Musical': 334,
         'Western': 167,
         'IMAX': 158,
         'Film-Noir': 87,
         '(no genres listed)': 34})

In [46]:
movies_to_replace = ['Western','IMAX','Film-Noir','(no genres listed)']

In [47]:
genres = set(g for G in movies_df['genres'] for g in G)
for g in genres:
    movies_df[g] = movies_df.genres.transform(lambda x: int(g in x))
    
movies_genres = movies_df.drop(columns=['movieId', 'title','genres','release_year'])
movies_genres

,Animation,IMAX,Romance,Children,Western,(no genres listed),Documentary,Musical,Action,Film-Noir,Sci-Fi,War,Fantasy,Drama,Comedy,Crime,Horror,Adventure,Mystery,Thriller
0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0
9738,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
9739,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9740,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [48]:
cutoff = 200
genres = {'other':0}
b = 0
for k, v in zip(keys, values):
    if v < cutoff:
        genres['other'] += v
    else:
        genres[k] = v

In [49]:
for g in genres:
    print(g)

other
Adventure
Animation
Children
Comedy
Fantasy
Romance
Drama
Action
Crime
Thriller
Horror
Mystery
Sci-Fi
War
Musical
Documentary


In [50]:
cutoff = 200
genres = {'other':0}
b = 0
for k, v in zip(keys, values):
    if v < cutoff:
        genres['other'] += v
    else:
        genres[k] = v
        

for g in genres:        
    movies_df[g] = movies_df.genres.transform(lambda x: int(g in x))    
movies_genres = movies_df.drop(columns=['movieId', 'title','genres','release_year'])
movies_genres

,Animation,IMAX,Romance,Children,Western,(no genres listed),Documentary,Musical,Action,Film-Noir,...,War,Fantasy,Drama,Comedy,Crime,Horror,Adventure,Mystery,Thriller,other
0,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
9738,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
9739,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9740,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
import numpy as np
def cumsum_bincount(a):  
    # Append 0 & look for a [0,1] pattern. Form a binned array based off 1s groups
    ids = a*(np.diff(np.r_[0,a])==1).cumsum()

    # Get the bincount, index into the count with ids and finally mask out 0s
    return a*np.bincount(ids)[ids]

In [52]:
movies_genres.value_counts()

Animation  IMAX  Romance  Children  Western  (no genres listed)  Documentary  Musical  Action  Film-Noir  Sci-Fi  War  Fantasy  Drama  Comedy  Crime  Horror  Adventure  Mystery  Thriller  other
0          0     0        0         0        0                   0            0        0       0          0       0    0        1      0       0      0       0          0        0         0        1053
                                                                                                                                0      1       0      0       0          0        0         0         946
                                                                                                                                1      1       0      0       0          0        0         0         435
                 1        0         0        0                   0            0        0       0          0       0    0        0      1       0      0       0          0        0         0         36

In [53]:
genres

{'other': 446,
 'Adventure': 1263,
 'Animation': 611,
 'Children': 664,
 'Comedy': 3756,
 'Fantasy': 779,
 'Romance': 1596,
 'Drama': 4361,
 'Action': 1828,
 'Crime': 1199,
 'Thriller': 1894,
 'Horror': 978,
 'Mystery': 573,
 'Sci-Fi': 980,
 'War': 382,
 'Musical': 334,
 'Documentary': 440}

In [54]:
movies_genres.head()

,Animation,IMAX,Romance,Children,Western,(no genres listed),Documentary,Musical,Action,Film-Noir,...,War,Fantasy,Drama,Comedy,Crime,Horror,Adventure,Mystery,Thriller,other
0,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [55]:
model = DBSCAN(eps = 0.5, metric = 'l2', min_samples = 15)
model.fit(movies_genres)

DBSCAN(metric='l2', min_samples=15)

In [56]:
len(model.components_)

7514

In [57]:
pd.Series(model.labels_).value_counts()

-1     2228
 10    1053
 3      946
 24     435
 1      363
       ... 
 79      15
 86      15
 82      15
 81      15
 55      15
Name: count, Length: 101, dtype: int64

In [58]:
movies_genres['label'] = model.labels_ + 1
movies_genres.loc[movies_genres['label'] == 2]

,Animation,IMAX,Romance,Children,Western,(no genres listed),Documentary,Musical,Action,Film-Noir,...,Fantasy,Drama,Comedy,Crime,Horror,Adventure,Mystery,Thriller,other,label
2,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
6,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
35,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
57,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
60,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9186,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
9370,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
9399,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
9466,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2


In [59]:
vectors = movies_genres.groupby('label').mean()
counts = movies_genres.groupby('label')['label'].count()

for cluster in range(movies_genres['label'].max()+1):
    print(f'\nCluster {cluster}, count: {counts[cluster]}')
    temp_df = vectors.transpose()[cluster].sort_values(ascending=False)
    identifying_categories = temp_df[temp_df > 0.9]
    [print(x) for x in identifying_categories.index] if len(identifying_categories > 0) else print('()')
    print('\n')
    print(vectors.transpose()[cluster].sort_values(ascending=False).head())
    print('\n' + '-'*40)


Cluster 0, count: 2228
()


Drama        0.374327
Comedy       0.362208
Action       0.328995
Adventure    0.321364
Thriller     0.279623
Name: 0, dtype: float64

----------------------------------------

Cluster 1, count: 24
Children
Adventure
Fantasy


Children     1.0
Adventure    1.0
Fantasy      1.0
Animation    0.0
War          0.0
Name: 1, dtype: float64

----------------------------------------

Cluster 2, count: 363
Romance
Comedy


Romance      1.0
Comedy       1.0
Animation    0.0
War          0.0
Thriller     0.0
Name: 2, dtype: float64

----------------------------------------

Cluster 3, count: 276
Romance
Comedy
Drama


Romance      1.0
Comedy       1.0
Drama        1.0
Animation    0.0
War          0.0
Name: 3, dtype: float64

----------------------------------------

Cluster 4, count: 946
Comedy


Comedy       1.0
Animation    0.0
War          0.0
Thriller     0.0
Mystery      0.0
Name: 4, dtype: float64

----------------------------------------

Cluster 5, count: 66


In [60]:
counts.sum()

9742

In [61]:
movies_genres.head(10)

,Animation,IMAX,Romance,Children,Western,(no genres listed),Documentary,Musical,Action,Film-Noir,...,Fantasy,Drama,Comedy,Crime,Horror,Adventure,Mystery,Thriller,other,label
0,1,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
3,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,4
5,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,5
6,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
7,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,6
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,7
9,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,8


In [62]:
ratings = ratings_df.groupby(['movieId', 'userId'])['rating'].mean()
ratings

movieId  userId
1        1         4.0
         5         4.0
         7         4.5
         15        2.5
         17        4.5
                  ... 
193581   184       4.0
193583   184       3.5
193585   184       3.5
193587   184       3.5
193609   331       4.0
Name: rating, Length: 100836, dtype: float64

In [63]:
ratings_df = ratings.to_frame()
ratings_df = ratings_df.reset_index()

In [64]:
ratings_df.head(10)

,movieId,userId,rating
0,1,1,4.0
1,1,5,4.0
2,1,7,4.5
3,1,15,2.5
4,1,17,4.5
5,1,18,3.5
6,1,19,4.0
7,1,21,3.5
8,1,27,3.0
9,1,31,5.0


In [65]:
movies_genres['movieId'] = movies_df['movieId']
movies_genres

,Animation,IMAX,Romance,Children,Western,(no genres listed),Documentary,Musical,Action,Film-Noir,...,Drama,Comedy,Crime,Horror,Adventure,Mystery,Thriller,other,label,movieId
0,1,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,2
2,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,3
3,0,0,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,3,4
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,193581
9738,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,193583
9739,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,11,193585
9740,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,193587


In [66]:
movies_genres.isna().sum()

Animation             0
IMAX                  0
Romance               0
Children              0
Western               0
(no genres listed)    0
Documentary           0
Musical               0
Action                0
Film-Noir             0
Sci-Fi                0
War                   0
Fantasy               0
Drama                 0
Comedy                0
Crime                 0
Horror                0
Adventure             0
Mystery               0
Thriller              0
other                 0
label                 0
movieId               0
dtype: int64

In [67]:
ratings.isna().sum()

0

In [68]:
df2 = pd.merge(ratings_df, movies_genres[['label', 'movieId']], on='movieId', how = 'inner')
df2.head(10)

,movieId,userId,rating,label
0,1,1,4.0,0
1,1,5,4.0,0
2,1,7,4.5,0
3,1,15,2.5,0
4,1,17,4.5,0
5,1,18,3.5,0
6,1,19,4.0,0
7,1,21,3.5,0
8,1,27,3.0,0
9,1,31,5.0,0


In [69]:
df2.isna().sum()

movieId    0
userId     0
rating     0
label      0
dtype: int64

In [70]:
df2 = df2.dropna()

In [71]:
df2.isna().sum()

movieId    0
userId     0
rating     0
label      0
dtype: int64

In [72]:
X = df2.drop('rating', axis=1)
y = df2['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
nn_model = tf.keras.Sequential()
nn_model.add(tf.keras.layers.Dense(units=1, activation = 'relu', input_dim=len(X.columns)))
nn_model.add(tf.keras.layers.Dense(units=16, activation = 'relu'))
nn_model.add(tf.keras.layers.Dense(units=16, activation = 'relu'))
nn_model.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
nn_model.compile(loss="MSE", optimizer="adam", metrics=["mse", "mae", 'accuracy'])
fit_model = nn_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
 629/2364 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.0282 - loss: 9.8533 - mae: 2.9608 - mse: 9.8533

KeyboardInterrupt: 

In [ ]:
r2_score(y_test, nn_model.predict(X_test))

In [ ]:
mean_absolute_error(y_test, nn_model.predict(X_test))